In [37]:
import numpy as np
import pandas as pd
import geocoder
import json 
from pandas.io.json import json_normalize 
from geopy.geocoders import Nominatim
import requests
import matplotlib.cm as cm
import matplotlib.colors as colors
from sklearn.cluster import KMeans 
import folium
print('Downloaded Library')

Downloaded Library


### I could not find any proper dataset of zipcodes in Nigeria to crape off the internet. So, I made my own and exported it as a csv file and read it into a DataFrame

In [2]:
Amsterdam = pd.read_csv('C://Users//jaklo/Downloads/AMS.csv')
print(Amsterdam.shape)
Amsterdam.head(10)

(591, 7)


,subject,region_name,regio_type,region_code,lat,lon,ninhabitants
0,Amsterdam,Amsterdam,Gemeente,NaN,NaN,NaN,833624
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Burgwallen-Oude Zijde,Amsterdam,Wijk,WK036300,52.37256586,4.896942721,4280
5,Kop Zeedijk,Amsterdam,Buurt,BU03630000,52.3759366,4.900279874,1020
6,Oude Kerk e.o.,Amsterdam,Buurt,BU03630001,52.37379307,4.896155554,670
7,Burgwallen Oost,Amsterdam,Buurt,BU03630002,52.37281532,4.898757438,1610
8,Nes e.o.,Amsterdam,Buurt,BU03630003,52.37086072,4.893727819,370
9,BG-terrein e.o.,Amsterdam,Buurt,BU03630004,52.36954598,4.895583642,605


In [3]:
Amsterdam = Amsterdam[Amsterdam['region_name'] == 'Amsterdam']
print(Amsterdam.shape)
Amsterdam.head()

(579, 7)


,subject,region_name,regio_type,region_code,lat,lon,ninhabitants
0,Amsterdam,Amsterdam,Gemeente,NaN,NaN,NaN,833624
4,Burgwallen-Oude Zijde,Amsterdam,Wijk,WK036300,52.37256586,4.896942721,4280
5,Kop Zeedijk,Amsterdam,Buurt,BU03630000,52.3759366,4.900279874,1020
6,Oude Kerk e.o.,Amsterdam,Buurt,BU03630001,52.37379307,4.896155554,670
7,Burgwallen Oost,Amsterdam,Buurt,BU03630002,52.37281532,4.898757438,1610


In [4]:
print(Amsterdam.dtypes)

subject         object
region_name     object
regio_type      object
region_code     object
lat             object
lon             object
ninhabitants    object
dtype: object


In [5]:
print(Amsterdam.isnull().sum())
Amsterdam[Amsterdam.lat.isnull()]

subject         0
region_name     0
regio_type      0
region_code     1
lat             4
lon             4
ninhabitants    0
dtype: int64


,subject,region_name,regio_type,region_code,lat,lon,ninhabitants
0,Amsterdam,Amsterdam,Gemeente,NaN,NaN,NaN,833624
206,Zeeburgereiland/Nieuwe Diep,Amsterdam,Wijk,WK036334,NaN,NaN,1215
213,Nieuwe Diep/Diemerpark,Amsterdam,Buurt,BU03633404,NaN,NaN,85
430,Durgerdam,Amsterdam,Buurt,BU03637303,NaN,NaN,500


In [6]:
Amsterdam.dropna(inplace=True)
print(Amsterdam.shape)
Amsterdam.head()

(575, 7)


,subject,region_name,regio_type,region_code,lat,lon,ninhabitants
4,Burgwallen-Oude Zijde,Amsterdam,Wijk,WK036300,52.37256586,4.896942721,4280
5,Kop Zeedijk,Amsterdam,Buurt,BU03630000,52.3759366,4.900279874,1020
6,Oude Kerk e.o.,Amsterdam,Buurt,BU03630001,52.37379307,4.896155554,670
7,Burgwallen Oost,Amsterdam,Buurt,BU03630002,52.37281532,4.898757438,1610
8,Nes e.o.,Amsterdam,Buurt,BU03630003,52.37086072,4.893727819,370


In [8]:
#Select rows with ninhabitants greater than 4000
Amsterdam.rename(columns = {'ninhabitants':'Population'}, inplace=True)
Amsterdam['Population'] = Amsterdam.Population.astype(int)
Amsterdam = Amsterdam[Amsterdam['Population'] > 4000]
Amsterdam.shape

(107, 7)

In [9]:
Amsterdam = Amsterdam[['subject', 'region_code', 'lat', 'lon']]
Amsterdam.head()

,subject,region_code,lat,lon
4,Burgwallen-Oude Zijde,WK036300,52.37256586,4.896942721
19,Grachtengordel-West,WK036302,52.37370923,4.886836099
24,Grachtengordel-Zuid,WK036303,52.36482059,4.89334883
32,Nieuwmarkt/Lastage,WK036304,52.37243584,4.905641464
42,Haarlemmerbuurt,WK036305,52.38417069,4.890581825


In [10]:
Amsterdam.rename(columns = {'subject':'Neighborhoods',
                            'region_code':'Region Code',
                            'lat':'Latitude',
                            'lon':'Longitude'}, inplace=True)
Amsterdam.reset_index(inplace=True)
Amsterdam = Amsterdam[['Neighborhoods', 'Region Code', 'Latitude', 'Longitude']]
Amsterdam.head()

,Neighborhoods,Region Code,Latitude,Longitude
0,Burgwallen-Oude Zijde,WK036300,52.37256586,4.896942721
1,Grachtengordel-West,WK036302,52.37370923,4.886836099
2,Grachtengordel-Zuid,WK036303,52.36482059,4.89334883
3,Nieuwmarkt/Lastage,WK036304,52.37243584,4.905641464
4,Haarlemmerbuurt,WK036305,52.38417069,4.890581825


In [11]:
Amsterdam['Latitude'] = Amsterdam.Latitude.astype(float)
Amsterdam['Longitude'] = Amsterdam.Longitude.astype(float)
print(Amsterdam.shape)
Amsterdam.dtypes

(107, 4)


Neighborhoods     object
Region Code       object
Latitude         float64
Longitude        float64
dtype: object

## Explore and Cluster Neighborhoods of Amsterdam
### Using FourSquare API to explore neighborhoods

In [12]:
CLIENT_ID = '3KH1DV2LCDYXWX1PWZWP1VYVJWFKRHIXICVWDOR0XY0CFBHI'
CLIENT_SECRET = 'KSGZ3ZC5PEREAZTBE5I0WBMPMEBHUN041ATC5YSINJAT3ZQD'
VERSION = '20180605'

In [13]:
neighborhood_latitude = Amsterdam.iloc[0,2]
neighborhood_longitude = Amsterdam.iloc[0,3]
neighborhood_name = Amsterdam.iloc[0,0]
print('The Latitude and Longitude values of {} are {},{}.'.format(neighborhood_name,
                                                                 neighborhood_latitude,
                                                                 neighborhood_longitude))

The Latitude and Longitude values of Burgwallen-Oude Zijde are 52.37256586,4.896942721.


In [14]:
LIMIT = 100
radius = 500
url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
    CLIENT_ID,
    CLIENT_SECRET,
    VERSION,
    neighborhood_latitude,
    neighborhood_longitude,
    radius,
    LIMIT)
url

'https://api.foursquare.com/v2/venues/explore?&client_id=3KH1DV2LCDYXWX1PWZWP1VYVJWFKRHIXICVWDOR0XY0CFBHI&client_secret=KSGZ3ZC5PEREAZTBE5I0WBMPMEBHUN041ATC5YSINJAT3ZQD&v=20180605&ll=52.37256586,4.896942721&radius=500&limit=100'

In [15]:
#Get requests and examine results
results = requests.get(url).json()
results

{'meta': {'code': 200, 'requestId': '5efb48d7f9762512a17aa583'},
 'response': {'suggestedFilters': {'header': 'Tap to show:',
   'filters': [{'name': 'Open now', 'key': 'openNow'}]},
  'headerLocation': 'De Wallen',
  'headerFullLocation': 'De Wallen, Amsterdam',
  'headerLocationGranularity': 'neighborhood',
  'totalResults': 129,
  'suggestedBounds': {'ne': {'lat': 52.377065864500004,
    'lng': 4.904299678514492},
   'sw': {'lat': 52.3680658555, 'lng': 4.889585763485509}},
  'groups': [{'type': 'Recommended Places',
    'name': 'recommended',
    'items': [{'reasons': {'count': 0,
       'items': [{'summary': 'This spot is popular',
         'type': 'general',
         'reasonName': 'globalInteractionReason'}]},
      'venue': {'id': '589e54ab2c55ec7cf2e2f707',
       'name': "Rosalia's Menagerie",
       'location': {'address': 'Kloveniersburgwal 20',
        'crossStreet': 'Koestraat',
        'lat': 52.3716780431221,
        'lng': 4.899174389497594,
        'labeledLatLngs': [{'

In [16]:
#Borrow get_category_type from Foursquare
def get_category_type(row):
    try:
        categories_list = row['categories']
    except: 
        categories_list = row['venue.categories']
    
    if len(categories_list)==0:
        return None
    else:
        return categories_list[0]['name']

In [17]:
#Clean json and structure ot into a pandas DataFrame 
venues = results['response']['groups'][0]['items']
nearby_venues = json_normalize(venues)

#Filter columns
filtered_columns = ['venue.name', 'venue.categories', 'venue.location.lat', 'venue.location.lng']

#Filter category for each row
nearby_venues['venues.categories'] = nearby_venues.apply(get_category_type, axis=1)

#Clean columns
nearby_venues.columns = [col.split(".")[-1] for col in nearby_venues.columns]

nearby_venues.head()

,count,items,referralId,categories,id,address,cc,city,country,crossStreet,...,lat,lng,neighborhood,postalCode,state,name,count,groups,id,categories
0,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-589e54ab2c55ec7cf2e2f707-0,"[{'id': '4bf58dd8d48988d11e941735', 'name': 'C...",589e54ab2c55ec7cf2e2f707,Kloveniersburgwal 20,NL,Amsterdam,Nederland,Koestraat,...,52.371678,4.899174,De Wallen,1012 CV,Noord-Holland,Rosalia's Menagerie,0,[],444960401,Cocktail Bar
1,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-51d98583498e78da6626be60-1,"[{'id': '4bf58dd8d48988d1e0931735', 'name': 'C...",51d98583498e78da6626be60,Oudekerksplein 27,NL,Amsterdam,Nederland,Oudezijds Voorburgwal,...,52.374043,4.898427,NaN,1012 GX,Noord-Holland,De Koffieschenkerij,0,[],NaN,Coffee Shop
2,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4a270666f964a520d88a1fe3-2,"[{'id': '4bf58dd8d48988d1fa931735', 'name': 'H...",4a270666f964a520d88a1fe3,Oudezijds Voorburgwal 197,NL,Amsterdam,Nederland,NaN,...,52.371093,4.895410,NaN,1012 EX,Noord-Holland,Sofitel Legend The Grand Amsterdam,0,[],408686723,Hotel
3,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4a270317f964a52075841fe3-3,"[{'id': '4bf58dd8d48988d186941735', 'name': 'L...",4a270317f964a52075841fe3,Pijlsteeg 31,NL,Amsterdam,Nederland,Dam,...,52.372301,4.895253,NaN,1012 HH,Noord-Holland,Wynand Fockink,0,[],NaN,Liquor Store
4,0,"[{'summary': 'This spot is popular', 'type': '...",e-0-4b9ff646f964a520f44d37e3-4,"[{'id': '4bf58dd8d48988d164941735', 'name': 'P...",4b9ff646f964a520f44d37e3,Dam,NL,Amsterdam,Nederland,NaN,...,52.372824,4.893702,Stadsdeel Centrum,1012 NP,Noord-Holland,Dam,0,[],NaN,Plaza


In [18]:
#How many Venues returned by Foursquare
print('{} venues were returned by Foursquare.'.format(nearby_venues.shape[0]))

100 venues were returned by Foursquare.


In [19]:
#Create Funtion to iterate the process with all neighborhoods
def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list = []
    for name, lat, lng in zip (names, latitudes, longitudes):
        print(name)
        
        #Create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
        
        #Make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        #Return only relevant info for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])
        
    
    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return (nearby_venues) 

In [23]:
Ams_venues = getNearbyVenues(names = Amsterdam['Neighborhoods'],
                              latitudes = Amsterdam['Latitude'],
                              longitudes = Amsterdam['Longitude']
                              )

Burgwallen-Oude Zijde
Grachtengordel-West
Grachtengordel-Zuid
Nieuwmarkt/Lastage
Haarlemmerbuurt
Jordaan
Elandsgrachtbuurt
De Weteringschans
Weesperbuurt/Plantage
Oostelijke Eilanden/Kadijken
Spaarndammer- en Zeeheldenbuurt
Staatsliedenbuurt
Frederik Hendrikbuurt
Da Costabuurt
Da Costabuurt Noord
Kinkerbuurt
Van Lennepbuurt
Helmersbuurt
Overtoomse Sluis
Cremerbuurt West
Oude Pijp
Sarphatiparkbuurt
Nieuwe Pijp
Zuid Pijp
Weesperzijde
Oosterparkbuurt
Dapperbuurt
Dapperbuurt Noord
Transvaalbuurt
Transvaalbuurt West
Transvaalbuurt Oost
Indische Buurt West
Noordwestkwadrant Indische buurt Zuid
Zuidwestkwadrant Indische buurt
Indische Buurt Oost
Noordoostkwadrant Indische buurt
Oostelijk Havengebied
IJburg West
Haveneiland Noordoost
Landlust
Bosleeuw
Landlust Zuid
Gibraltarbuurt
Erasmuspark
Erasmusparkbuurt West
De Kolenkit
Geuzenbuurt
Van Galenbuurt
Hoofdweg e.o.
Westindische Buurt
Hoofddorppleinbuurt
Willemspark
Museumkwartier
Stadionbuurt
Van Tuyllbuurt
Apollobuurt
IJburg Zuid
Haveneiland 

In [24]:
Ams_venues.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Burgwallen-Oude Zijde,52.372566,4.896943,Rosalia's Menagerie,52.371678,4.899174,Cocktail Bar
1,Burgwallen-Oude Zijde,52.372566,4.896943,De Koffieschenkerij,52.374043,4.898427,Coffee Shop
2,Burgwallen-Oude Zijde,52.372566,4.896943,Sofitel Legend The Grand Amsterdam,52.371093,4.895410,Hotel
3,Burgwallen-Oude Zijde,52.372566,4.896943,Wynand Fockink,52.372301,4.895253,Liquor Store
4,Burgwallen-Oude Zijde,52.372566,4.896943,Dam,52.372824,4.893702,Plaza


In [26]:
Ams_venues.groupby('Neighborhood').count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
Apollobuurt,20,20,20,20,20,20
Banne Buiksloot,10,10,10,10,10,10
"Bijlmer Centrum (D,F,H)",47,47,47,47,47,47
"Bijlmer Oost (E,G,K)",24,24,24,24,24,24
Bosleeuw,50,50,50,50,50,50
Buikslotermeer,31,31,31,31,31,31
Buitenveldert-Oost,15,15,15,15,15,15
Buitenveldert-West,12,12,12,12,12,12
Burgwallen-Oude Zijde,100,100,100,100,100,100


In [27]:
#Find out how many unique categories curated from returned venues
print('There are {} unique venues.'.format(len(Ams_venues['Venue Category'].unique())))

There are 294 unique venues.


In [28]:
#Analysis of each Neighborhood
Ams_onehot = pd.get_dummies(Ams_venues[['Venue Category']], prefix="", prefix_sep="")
Ams_onehot['Neighborhood'] = Ams_venues['Neighborhood']
#Move column to first column
fixed_columns = [Ams_onehot.columns[-1]] + list(Ams_onehot.columns[:-1])
Ams_onehot = Ams_onehot[fixed_columns]
Ams_onehot.head()

,Zoo Exhibit,Accessories Store,Afghan Restaurant,African Restaurant,American Restaurant,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,Art Museum,...,Udon Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Water Park,Windmill,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Zoo
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [29]:
Ams_onehot.shape

(4088, 294)

In [31]:
#Group rows by neighbourhood and by taking the mean of the frequency of occurence of each category
Ams_grouped = Ams_onehot.groupby('Neighborhood').mean().reset_index()
Ams_grouped.head()

,Neighborhood,Zoo Exhibit,Accessories Store,Afghan Restaurant,African Restaurant,American Restaurant,Aquarium,Arcade,Argentinian Restaurant,Art Gallery,...,Udon Restaurant,Vegetarian / Vegan Restaurant,Vietnamese Restaurant,Water Park,Windmill,Wine Bar,Wine Shop,Women's Store,Yoga Studio,Zoo
0,Apollobuurt,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.05,...,0.0,0.00,0.0,0.0,0.0,0.0,0.05,0.0,0.0,0.0
1,Banne Buiksloot,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
2,"Bijlmer Centrum (D,F,H)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
3,"Bijlmer Oost (E,G,K)",0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.00,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0
4,Bosleeuw,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.00,...,0.0,0.06,0.0,0.0,0.0,0.0,0.00,0.0,0.0,0.0


In [33]:
#Write function to sort venues in descending order
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [36]:
num_top_venues = 5

indicators = ['st', 'nd', 'rd']

#Create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))
        
#Create a new dataFrame
neighborhoods_venues_sorted = pd.DataFrame(columns = columns)
neighborhoods_venues_sorted['Neighborhood'] = Ams_grouped['Neighborhood']

for ind in np.arange(Ams_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(Ams_grouped.iloc[ind,:], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Apollobuurt,Hotel,Baby Store,Health Food Store,Italian Restaurant,Juice Bar
1,Banne Buiksloot,Bus Stop,Supermarket,Park,Turkish Restaurant,Restaurant
2,"Bijlmer Centrum (D,F,H)",Sandwich Place,Chinese Restaurant,Clothing Store,Bakery,Supermarket
3,"Bijlmer Oost (E,G,K)",Drugstore,Shopping Mall,Supermarket,South American Restaurant,Snack Place
4,Bosleeuw,Bakery,Vegetarian / Vegan Restaurant,Supermarket,Turkish Restaurant,Tram Station


### Cluster Neighborhoods using K-Means

In [38]:
#Set number of clusters
kclusters = 5

Ams_grouped_clustering = Ams_grouped.drop('Neighborhood', 1)

#run k-means clustering
kmeans = KMeans (n_clusters = kclusters, random_state=0).fit(Ams_grouped_clustering)

#Check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]

array([1, 2, 1, 0, 1, 1, 1, 1, 1, 1])

In [44]:
#Note: Clustering Labels have been added.
Amsterdam.rename(columns={'Neighborhoods':'Neighborhood'}, inplace=True)

Ams_merged = Amsterdam

#Merge Ams_grouped with Amsterdam to add latitude and longitude for each neighborhood
Ams_merged = Ams_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')
Ams_merged.head()

,Neighborhood,Region Code,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Burgwallen-Oude Zijde,WK036300,52.372566,4.896943,1,Bar,Coffee Shop,Hotel,Brewery,Burger Joint
1,Grachtengordel-West,WK036302,52.373709,4.886836,1,Bar,Hotel,Café,Sandwich Place,French Restaurant
2,Grachtengordel-Zuid,WK036303,52.364821,4.893349,1,Italian Restaurant,Restaurant,Coffee Shop,Hotel,Café
3,Nieuwmarkt/Lastage,WK036304,52.372436,4.905641,1,Bar,Hotel,Hostel,Coffee Shop,Cocktail Bar
4,Haarlemmerbuurt,WK036305,52.384171,4.890582,1,Bar,Café,Restaurant,Coffee Shop,Organic Grocery


In [45]:
from geopy.exc import GeocoderTimedOut
from time import sleep
address = 'Amsterdam, NL'
geolocator = Nominatim(user_agent='ny_explorer')
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
sleep(1)

print('The geographical coordinates of Amsterdam are {}, {}.'.format(latitude, longitude))

The geographical coordinates of Amsterdam are 52.3727598, 4.8936041.


In [46]:
#Visualize resulting clusters
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

#set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]

colors_array = cm.rainbow(np.linspace(0,1,len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

In [47]:
#add markers to the map
markers_colors = []

for lat,lon, poi, cluster in zip(Ams_merged['Latitude'], Ams_merged['Longitude'], Ams_merged['Neighborhood'], Ams_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + 'Cluster' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)

map_clusters

### Examine Clusters

In [50]:
#Cluster 1
Ams_merged.loc[Ams_merged['Cluster Labels']==0, Ams_merged.columns[[0] + list(range(5, Ams_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
83,Osdorp-Oost,Turkish Restaurant,Shopping Mall,Supermarket,Drugstore,Clothing Store
84,Wildeman,Bakery,Supermarket,Clothing Store,Hotel,Snack Place
85,Osdorp-Midden,Turkish Restaurant,Ice Cream Shop,Snack Place,Bookstore,Sushi Restaurant
86,Zuidwestkwadrant Osdorp Zuid,Supermarket,Mexican Restaurant,Tram Station,Park,Drugstore
89,Middelveldsche Akerpolder,Tram Station,Pizza Place,Boat or Ferry,Supermarket,Bakery
91,De Aker Oost,Tram Station,Pizza Place,Cheese Shop,Pharmacy,Bed & Breakfast
93,Overtoomse Veld,Supermarket,Hotel,Farm,Nightclub,Restaurant
94,Overtoomse Veld Noord,Supermarket,Nightclub,Hotel,Tram Station,Bus Stop
97,Slotervaart Zuid,Coffee Shop,Thai Restaurant,Burger Joint,Chinese Restaurant,Supermarket
103,"Bijlmer Oost (E,G,K)",Drugstore,Shopping Mall,Supermarket,South American Restaurant,Snack Place


In [56]:
#Cluster 2
Ams_merged.loc[Ams_merged['Cluster Labels']==1, Ams_merged.columns[[0] + list(range(5, Ams_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
0,Burgwallen-Oude Zijde,Bar,Coffee Shop,Hotel,Brewery,Burger Joint
1,Grachtengordel-West,Bar,Hotel,Café,Sandwich Place,French Restaurant
2,Grachtengordel-Zuid,Italian Restaurant,Restaurant,Coffee Shop,Hotel,Café
3,Nieuwmarkt/Lastage,Bar,Hotel,Hostel,Coffee Shop,Cocktail Bar
4,Haarlemmerbuurt,Bar,Café,Restaurant,Coffee Shop,Organic Grocery
5,Jordaan,Bar,Hotel,Café,Italian Restaurant,Furniture / Home Store
6,Elandsgrachtbuurt,Bar,Restaurant,Furniture / Home Store,Hotel,Coffee Shop
7,De Weteringschans,Bar,Coffee Shop,Hotel,Restaurant,Café
8,Weesperbuurt/Plantage,Zoo Exhibit,Hotel,Restaurant,Bar,Café
9,Oostelijke Eilanden/Kadijken,History Museum,Café,Gym / Fitness Center,Ice Cream Shop,Snack Place


In [52]:
#Cluster 3
Ams_merged.loc[Ams_merged['Cluster Labels']==2, Ams_merged.columns[[0] + list(range(5, Ams_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
34,Indische Buurt Oost,Distillery,Bus Stop,Pool,Park,Gym / Fitness Center
69,Tuindorp Oostzaan,Bus Stop,Chinese Restaurant,Gym Pool,Snack Place,Baseball Field
72,Waterlandpleinbuurt,Bus Stop,Supermarket,Discount Store,Grocery Store,Indonesian Restaurant
73,Werengouw Midden,Bus Stop,Snack Place,Bakery,Indonesian Restaurant,Convenience Store
75,Banne Buiksloot,Bus Stop,Supermarket,Park,Turkish Restaurant,Restaurant
80,Geuzenveld,Bus Stop,Supermarket,Tram Station,Convenience Store,Turkish Restaurant
81,Buurt 8,Bus Stop,Tram Station,Turkish Restaurant,Drugstore,Supermarket
82,Buurt 9,Bus Stop,Park,Snack Place,Drugstore,Grocery Store
102,Venserpolder West,Turkish Restaurant,Bus Stop,Scenic Lookout,Athletics & Sports,Market


In [54]:
#Cluster 4
Ams_merged.loc[Ams_merged['Cluster Labels']==3, Ams_merged.columns[[0] + list(range(5, Ams_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
71,Oostzanerwerf,Soccer Field,Creperie,Bus Stop,Pier,Empanada Restaurant
92,Slotervaart Noord,Asian Restaurant,Moroccan Restaurant,Soccer Field,Martial Arts Dojo,Bus Stop


In [55]:
#Cluster 5
Ams_merged.loc[Ams_merged['Cluster Labels']==4, Ams_merged.columns[[0] + list(range(5, Ams_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue
90,De Aker West,Tram Station,Harbor / Marina,Museum,Trail,Zoo
